In [457]:
import pandas as pd

df = pd.read_csv(r"..\data\raw\nutritional-facts.csv")
df.fillna(0, inplace=True)
df.head()

,Food Name,Category Name,Calcium,Calories,Carbs,Cholesterol,Copper,Fats,Fiber,Folate,...,Vitamin D,Vitamin E,Vitamin K,Omega-3 - ALA,Omega-6 - Eicosadienoic acid,Omega-6 - Gamma-linoleic acid,Omega-3 - Eicosatrienoic acid,Omega-6 - Dihomo-gamma-linoleic acid,Omega-6 - Linoleic acid,Omega-6 - Arachidonic acid
0,Apple,Fruits,0.006,52.0,14.0,0.0,0.00003,0.17,2.4,0.000003,...,0.0,0.00018,2.200000e-06,0.00,0.0,0.00,0.0,0.0,0.0,0.0
1,Apricot,Fruits,0.013,48.0,11.0,0.0,0.00008,0.39,2.0,0.000009,...,0.0,0.00089,3.300000e-06,0.00,0.0,0.00,0.0,0.0,0.0,0.0
2,Dried fruit,Fruits,0.055,241.0,63.0,0.0,0.00034,0.51,7.3,0.000010,...,0.0,0.00430,3.100000e-06,0.00,0.0,0.00,0.0,0.0,0.0,0.0
3,Avocado,Fruits,0.012,160.0,8.5,0.0,0.00019,15.00,6.7,0.000081,...,0.0,0.00210,2.100000e-05,0.11,0.0,0.02,0.0,0.0,0.0,0.0
4,Banana,Fruits,0.005,89.0,23.0,0.0,0.00008,0.33,2.6,0.000020,...,0.0,0.00010,5.000000e-07,0.00,0.0,0.00,0.0,0.0,0.0,0.0


In [458]:
import json, datetime

with open('../data/raw/food-seasonality.json', 'r') as f:
    seasonality_data = json.load(f)

def find_seasonal_food(seasonality_data: any, month: str = ""):
    fruits = []
    vegetables = []

    if month == "":
        current_month = datetime.datetime.now().strftime("%m")
    else:
        current_month = month

    seasonal_foods = seasonality_data["Italy"].get(current_month, [])

    for food in seasonal_foods:
        food_category = df[df['Food Name'] == food]['Category Name'].values
        if len(food_category) > 0:
            if food_category[0] == "Fruits":
                fruits.append(food)
            else:
                vegetables.append(food)

    return fruits, vegetables

fruits, vegetables = find_seasonal_food(seasonality_data)
print("Seasonal fruits: ", fruits)
print("Seasonal vegetables: ", vegetables)

Seasonal fruits:  ['Orange', 'Clementine', 'Kiwifruit', 'Mandarin orange', 'Apple', 'Pear', 'Grapefruit']
Seasonal vegetables:  ['Artichoke', 'Carrot', 'Broccoli', 'Cauliflower', 'Cabbage', 'Chicory', 'Potato', 'Radicchio', 'Turnip', 'Spinach', 'Pumpkin']


In [459]:
def find_food_nutritional_info(food_name: str, only_numbers: bool = True):
    food_info = df[df['Food Name'] == food_name]
    if len(food_info) > 0:
        if only_numbers:
            return food_info.drop(columns=['Food Name', 'Category Name'])
        return food_info
    else:
        return "No information found for this food"
    
food_name = "Apple"
food_info = find_food_nutritional_info(food_name)
food_info

,Calcium,Calories,Carbs,Cholesterol,Copper,Fats,Fiber,Folate,Iron,Magnesium,...,Vitamin D,Vitamin E,Vitamin K,Omega-3 - ALA,Omega-6 - Eicosadienoic acid,Omega-6 - Gamma-linoleic acid,Omega-3 - Eicosatrienoic acid,Omega-6 - Dihomo-gamma-linoleic acid,Omega-6 - Linoleic acid,Omega-6 - Arachidonic acid
0,0.006,52.0,14.0,0.0,0.00003,0.17,2.4,0.000003,0.00012,0.005,...,0.0,0.00018,0.000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [460]:
def compute_energy_density_per_100g(food_name):
    # Low energy density (< 1.5 kcal/g)
    # Medium energy density (1.5 - 2.5 kcal/g)
    # High energy density (> 2.5 kcal/g)
    food_info = find_food_nutritional_info(food_name)
    if isinstance(food_info, str):
        return food_info
    energy_density = food_info['Calories'].values[0] / 100

    if energy_density < 1.5:
        return ("Low", energy_density)
    elif 1.5 <= energy_density <= 2.5:
        return ("Medium", energy_density)
    else:
        return ("High", energy_density)

energy_density = compute_energy_density_per_100g(food_name="Ricotta")
energy_density

('Medium', np.float64(1.74))

In [461]:
import numpy as np
from numpy.linalg import norm

def find_similar_food(df: pd.DataFrame, food_name: str, n: int = 1, same_category: bool = True, low_density_food: bool = True):
    similar_foods = []

    if same_category:
        food_info = df[df['Food Name'] == food_name]
        if food_info.empty:
            return f"No information found for food: {food_name}"
        
        category = food_info['Category Name'].values[0]
        food_A = find_food_nutritional_info(food_name, only_numbers=True).to_numpy().flatten()
        
        for food in df[df['Category Name'] == category]['Food Name']:
            if food == food_name:
                continue
            food_B = find_food_nutritional_info(food, only_numbers=True).to_numpy().flatten()
            norm_A = norm(food_A)
            norm_B = norm(food_B)

            if norm_A == 0 or norm_B == 0:
                similarity = 0
            else:
                similarity = np.dot(food_A, food_B) / (norm_A * norm_B)
            similar_foods.append((food, similarity))

    if low_density_food == False or compute_energy_density_per_100g(food_name)[0] == "Low":
        similar_foods = sorted(similar_foods, key=lambda x: x[1], reverse=True)
    else:
        similar_foods = sorted(similar_foods, key=lambda x: x[1], reverse=True)
        similar_foods = sorted(similar_foods, key=lambda x: (compute_energy_density_per_100g(x[0])[1], x[1]))
        similar_foods = [(food, similarity, compute_energy_density_per_100g(food)[0]) for food, similarity in similar_foods]
        
    return similar_foods

similar_foods = find_similar_food(df, food_name="Apple")
print(similar_foods[0:3])

[('Blueberry', np.float64(0.9988352415031658)), ('Pineapple', np.float64(0.9970988211193896)), ('Honeydew', np.float64(0.9891949873765672))]


In [462]:
# how to print low density foods
low_density_foods = []
for food in df['Food Name']:
    density = compute_energy_density_per_100g(food)
    if density[0] == "Low":
        low_density_foods.append((food, density[1]))

In [463]:
def filter_by_diet_and_intolerances(df, diet='omnivore', gluten_free=False, dairy_free=False):
    # Create a copy of the dataframe
    filtered_df = df.copy()
    
    # Define categories to exclude based on diet
    exclude_categories = []
    if diet == 'vegan':
        exclude_categories.extend(['Dairy', 'Lactose-Free Dairy', 'Cured Meat', 'Red Meat', 'White Meat', 'Seafood', 'Eggs'])
    elif diet == 'vegetarian':
        exclude_categories.extend(['Cured Meat', 'Red Meat', 'White Meat', 'Seafood'])
    
    # Add dairy categories if dairy-free
    if dairy_free:
        exclude_categories.extend(['Dairy', 'Lactose-Free Dairy'])
    
    # Filter out excluded categories
    filtered_df = filtered_df[~filtered_df['Category Name'].isin(exclude_categories)]
    
    # Filter out gluten-containing foods if gluten-free
    if gluten_free:
        gluten_foods = ['Wheat Bread', 'Pasta']  # Add more gluten-containing foods as needed
        filtered_df = filtered_df[~filtered_df['Food Name'].isin(gluten_foods)]
    
    return filtered_df.reset_index(drop=True)

# ask the user and try the function
# diet = input("Please enter your diet (omnivore, vegetarian, vegan): ")
diet = "omnivore"
gluten_free = input("Are you gluten-free? (yes/no): ").lower() == 'yes'
dairy_free = input("Are you dairy-free? (yes/no): ").lower() == 'yes'

# save user input
user_input = {
    "diet": diet,
    "gluten_free": gluten_free,
    "dairy_free": dairy_free
}

filtered_df = filter_by_diet_and_intolerances(df, diet=diet, gluten_free=gluten_free, dairy_free=dairy_free)
filtered_df.head()

,Food Name,Category Name,Calcium,Calories,Carbs,Cholesterol,Copper,Fats,Fiber,Folate,...,Vitamin D,Vitamin E,Vitamin K,Omega-3 - ALA,Omega-6 - Eicosadienoic acid,Omega-6 - Gamma-linoleic acid,Omega-3 - Eicosatrienoic acid,Omega-6 - Dihomo-gamma-linoleic acid,Omega-6 - Linoleic acid,Omega-6 - Arachidonic acid
0,Apple,Fruits,0.006,52.0,14.0,0.0,0.00003,0.17,2.4,0.000003,...,0.0,0.00018,2.200000e-06,0.00,0.0,0.00,0.0,0.0,0.0,0.0
1,Apricot,Fruits,0.013,48.0,11.0,0.0,0.00008,0.39,2.0,0.000009,...,0.0,0.00089,3.300000e-06,0.00,0.0,0.00,0.0,0.0,0.0,0.0
2,Dried fruit,Fruits,0.055,241.0,63.0,0.0,0.00034,0.51,7.3,0.000010,...,0.0,0.00430,3.100000e-06,0.00,0.0,0.00,0.0,0.0,0.0,0.0
3,Avocado,Fruits,0.012,160.0,8.5,0.0,0.00019,15.00,6.7,0.000081,...,0.0,0.00210,2.100000e-05,0.11,0.0,0.02,0.0,0.0,0.0,0.0
4,Banana,Fruits,0.005,89.0,23.0,0.0,0.00008,0.33,2.6,0.000020,...,0.0,0.00010,5.000000e-07,0.00,0.0,0.00,0.0,0.0,0.0,0.0


In [464]:
def get_user_preferences(df: pd.DataFrame, user_input: dict, filtered_df: pd.DataFrame):
    
    # Define categories to collect preferences for
    categories = [
        "Grains", "Gluten-Free Grains", "Legumes", "Dairy", "Lactose-Free Dairy",
        "Cured Meat", "Red Meat", "White Meat", "Seafood",
        "Oils", "Sauces", "Eggs"
    ]

    if user_input.get("gluten_free") == "yes":
        categories.remove("Grains")

        default_preferences = {
            "Gluten-Free Grains": ["Rice"],
            "Legumes": ["Chickpeas", "Lentils"],
            "Dairy": ["Mozzarella",  "Ricotta"],
            "Lactose-Free Dairy": ["Parmigiano-Reggiano"],
            "Cured Meat": ["Mortadella", "Salami"],
            "Red Meat": ["Steak"],
            "White Meat": ["Chicken meat"],
            "Seafood": ["Tuna", "Salmon"],
            "Eggs": ["Eggs"],
            "Oils": ["Olive oil"],
            "Sauces": ["Tomato Sauce", "Salsa"]
        }
    elif user_input.get("dairy_free") == "yes":
        categories.remove("Dairy")

        default_preferences = {
            "Grains": ["Pasta", "Wheat Bread"],
            "Gluten-Free Grains": ["Rice"],
            "Legumes": ["Chickpeas", "Lentils"],
            "Lactose-Free Dairy": ["Parmigiano-Reggiano"],
            "Cured Meat": ["Mortadella", "Salami"],
            "Red Meat": ["Steak"],
            "White Meat": ["Chicken meat"],
            "Seafood": ["Tuna", "Salmon"],
            "Eggs": ["Eggs"],
            "Oils": ["Olive oil"],
            "Sauces": ["Tomato Sauce", "Salsa"]
        }
    else:
        default_preferences = {
            "Grains": ["Pasta", "Wheat Bread"],
            "Gluten-Free Grains": ["Rice"],
            "Legumes": ["Chickpeas", "Lentils"],
            "Dairy": ["Mozzarella",  "Ricotta"],
            "Lactose-Free Dairy": ["Parmigiano-Reggiano"],
            "Cured Meat": ["Mortadella", "Salami"],
            "Red Meat": ["Steak"],
            "White Meat": ["Chicken meat"],
            "Seafood": ["Tuna", "Salmon"],
            "Eggs": ["Eggs"],
            "Oils": ["Olive oil"],
            "Sauces": ["Tomato Sauce", "Salsa"]
        }
    
    preferences = {}
    
    print("Please select your preferred foods for each category.")
    print("Enter numbers separated by spaces, or press Enter to skip.")
    
    for category in categories:
        # Get foods for this category from filtered_df (respecting dietary restrictions)
        foods = filtered_df[filtered_df["Category Name"] == category]["Food Name"].tolist()
        
        if not foods:  # Skip empty categories (due to dietary restrictions)
            continue
            
        # Show options
        print(f"\n{category}:")
        for i, food in enumerate(foods, 1):
            print(f"{i}. {food}")
        
        # Get user input
        while True:
            try:
                choices = input(f"Select {category} (1-{len(foods)}): ").strip()
                if not choices:  # If no selection, include all foods from this category
                    preferences[category] = default_preferences[category]
                    break
                    
                # Convert input to list of integers
                selected = [int(x) for x in choices.split()]
                
                # Validate choices
                if all(1 <= x <= len(foods) for x in selected):
                    preferences[category] = [foods[i-1] for i in selected]
                    break
                else:
                    print("Invalid selection. Please try again.")
            except ValueError:
                print("Please enter valid numbers separated by spaces.")

    # Handle cold start: if no preferences were selected, use filtered_df
    if not preferences:
        return filtered_df
    
    # Combine all preferences into a single dataframe
    combined_preferences = pd.concat([filtered_df[filtered_df["Food Name"].isin(preferences[category])] 
                                    for category in preferences])
    
    # If combined_preferences is empty, fallback to filtered_df
    if combined_preferences.empty:
        return filtered_df
    
    # add the following categories without asking for preferences Sweets Breakfast, Baked Products Breakfast, Beverages, Lactose-Free Dairy Breakfast, Dairy Breakfast
    breakfast_categories = ["Sweets Breakfast", "Baked Products Breakfast", "Beverages", "Lactose-Free Dairy Breakfast", "Dairy Breakfast", "Nuts Breakfast", "Nuts", "Fast Foods", "Sweets"]
    breakfast_preferences = pd.concat([filtered_df[filtered_df["Category Name"] == category] for category in breakfast_categories])
    
    # Add breakfast preferences to combined_preferences
    combined_preferences = pd.concat([combined_preferences, breakfast_preferences])
        
    return combined_preferences.reset_index(drop=True)

user_preferences = get_user_preferences(df, user_input, filtered_df)
user_preferences

Please select your preferred foods for each category.
Enter numbers separated by spaces, or press Enter to skip.

Grains:
1. Wheat Bread
2. Couscous
3. Pasta

Gluten-Free Grains:
1. Buckwheat
2. Millet
3. Oat
4. Rice

Legumes:
1. Chickpeas
2. Lentil
3. Soybean
4. Tempeh
5. Hummus
6. Tofu
7. Bean

Dairy:
1. Cheese
2. Cottage cheese
3. Cream cheese
4. Ricotta
5. Roquefort
6. Goat cheese
7. Mozzarella

Lactose-Free Dairy:
1. Blue cheese
2. Brie
3. Camembert
4. Edam
5. Feta
6. Fontina
7. Gouda cheese
8. Gruyere cheese
9. Provolone
10. Romano cheese
11. Swiss cheese
12. Parmigiano-Reggiano
13. Margarine

Cured Meat:
1. Turkey ham
2. Luncheon meat
3. Mortadella
4. Salami
5. Italian sausage
6. Turkey bacon
7. Turkey sausage
8. Ham
9. Pork bacon

Red Meat:
1. Roast beef
2. Meatball
3. Pork belly
4. Pork
5. Ground beef
6. Goat
7. Steak
8. Beef

White Meat:
1. Chicken meat
2. Turkey meat
3. Ground turkey
4. Ground chicken
5. Lamb
6. Rabbit Meat

Seafood:
1. Anchovy
2. Carp
3. Cod
4. Fish sticks


,Food Name,Category Name,Calcium,Calories,Carbs,Cholesterol,Copper,Fats,Fiber,Folate,...,Vitamin D,Vitamin E,Vitamin K,Omega-3 - ALA,Omega-6 - Eicosadienoic acid,Omega-6 - Gamma-linoleic acid,Omega-3 - Eicosatrienoic acid,Omega-6 - Dihomo-gamma-linoleic acid,Omega-6 - Linoleic acid,Omega-6 - Arachidonic acid
0,Wheat Bread,Grains,0.165,313.0,56.0,0.000,0.00020,4.30,4.7,0.000086,...,0.000000e+00,0.00024,5.700000e-06,0.16,0.0,0.0,0.0,0.0,0.00,0.0
1,Couscous,Grains,0.008,112.0,23.0,0.000,0.00004,0.16,1.4,0.000015,...,0.000000e+00,0.00013,1.000000e-07,0.00,0.0,0.0,0.0,0.0,0.00,0.0
2,Pasta,Grains,0.006,131.0,25.0,0.033,0.00009,1.10,0.0,0.000064,...,0.000000e+00,0.00000,0.000000e+00,0.00,0.0,0.0,0.0,0.0,0.00,0.0
3,Rice,Gluten-Free Grains,0.010,130.0,28.0,0.000,0.00007,0.28,0.4,0.000058,...,0.000000e+00,0.00004,0.000000e+00,0.00,0.0,0.0,0.0,0.0,0.00,0.0
4,Chickpeas,Legumes,0.049,164.0,27.0,0.000,0.00035,2.60,7.6,0.000172,...,0.000000e+00,0.00035,4.000000e-06,0.00,0.0,0.0,0.0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,Praline,Sweets,0.043,485.0,60.0,0.000,0.00049,26.00,3.5,0.000006,...,0.000000e+00,0.00050,1.300000e-06,0.00,0.0,0.0,0.0,0.0,0.00,0.0
124,Mars,Sweets,0.090,475.0,72.0,0.012,0.00000,19.00,2.0,0.000000,...,0.000000e+00,0.00000,0.000000e+00,0.00,0.0,0.0,0.0,0.0,0.00,0.0
125,Snickers,Sweets,0.090,488.0,63.0,0.009,0.00000,24.00,1.9,0.000000,...,0.000000e+00,0.00000,0.000000e+00,0.00,0.0,0.0,0.0,0.0,0.00,0.0
126,Chocolate,Sweets,0.056,546.0,61.0,0.008,0.00100,31.00,7.0,0.000000,...,0.000000e+00,0.00054,8.100000e-06,0.09,0.0,0.0,0.0,0.0,0.95,0.0


In [465]:
def update_user_preferences_with_seasonal(user_preferences, df, seasonality_data):
    # Get current month
    current_month = datetime.datetime.now().strftime("%m")

    # Find seasonal fruits and vegetables
    seasonal_fruits, seasonal_vegetables = find_seasonal_food(seasonality_data, current_month)

    # Ask the user for their preferred seasonal fruits and vegetables
    print("Please select your preferred seasonal fruits and vegetables for this month.")
    print("Enter numbers separated by spaces, or press Enter to select all.")

    # Get user preferences for seasonal fruits
    print("\nSeasonal Fruits:")
    for i, fruit in enumerate(seasonal_fruits, 1):
        print(f"{i}. {fruit}")

    selected_fruits = input(f"Select fruits (1-{len(seasonal_fruits)}): ").strip()
    if selected_fruits:
        selected_fruits = [seasonal_fruits[int(i)-1] for i in selected_fruits.split()]
    else:
        selected_fruits = seasonal_fruits

    # Get user preferences for seasonal vegetables
    print("\nSeasonal Vegetables:")
    for i, vegetable in enumerate(seasonal_vegetables, 1):
        print(f"{i}. {vegetable}")

    selected_vegetables = input(f"Select vegetables (1-{len(seasonal_vegetables)}): ").strip()
    if selected_vegetables:
        selected_vegetables = [seasonal_vegetables[int(i)-1] for i in selected_vegetables.split()]
    else:
        selected_vegetables = seasonal_vegetables

    # Filter the user preferences dataframe to include only selected seasonal fruits and vegetables
    selected_fruits_df = df[df['Food Name'].isin(selected_fruits)]
    selected_vegetables_df = df[df['Food Name'].isin(selected_vegetables)]

    # Combine selected seasonal fruits and vegetables with user preferences
    user_preferences = pd.concat([user_preferences, selected_fruits_df, selected_vegetables_df]).drop_duplicates().reset_index(drop=True)

    return user_preferences

user_preferences = update_user_preferences_with_seasonal(user_preferences, df, seasonality_data)
user_preferences

Please select your preferred seasonal fruits and vegetables for this month.
Enter numbers separated by spaces, or press Enter to select all.

Seasonal Fruits:
1. Orange
2. Clementine
3. Kiwifruit
4. Mandarin orange
5. Apple
6. Pear
7. Grapefruit

Seasonal Vegetables:
1. Artichoke
2. Carrot
3. Broccoli
4. Cauliflower
5. Cabbage
6. Chicory
7. Potato
8. Radicchio
9. Turnip
10. Spinach
11. Pumpkin


,Food Name,Category Name,Calcium,Calories,Carbs,Cholesterol,Copper,Fats,Fiber,Folate,...,Vitamin D,Vitamin E,Vitamin K,Omega-3 - ALA,Omega-6 - Eicosadienoic acid,Omega-6 - Gamma-linoleic acid,Omega-3 - Eicosatrienoic acid,Omega-6 - Dihomo-gamma-linoleic acid,Omega-6 - Linoleic acid,Omega-6 - Arachidonic acid
0,Wheat Bread,Grains,0.165,313.0,56.0,0.000,0.00020,4.30,4.7,0.000086,...,0.0,0.00024,5.700000e-06,0.16,0.0,0.0,0.0,0.0,0.0,0.0
1,Couscous,Grains,0.008,112.0,23.0,0.000,0.00004,0.16,1.4,0.000015,...,0.0,0.00013,1.000000e-07,0.00,0.0,0.0,0.0,0.0,0.0,0.0
2,Pasta,Grains,0.006,131.0,25.0,0.033,0.00009,1.10,0.0,0.000064,...,0.0,0.00000,0.000000e+00,0.00,0.0,0.0,0.0,0.0,0.0,0.0
3,Rice,Gluten-Free Grains,0.010,130.0,28.0,0.000,0.00007,0.28,0.4,0.000058,...,0.0,0.00004,0.000000e+00,0.00,0.0,0.0,0.0,0.0,0.0,0.0
4,Chickpeas,Legumes,0.049,164.0,27.0,0.000,0.00035,2.60,7.6,0.000172,...,0.0,0.00035,4.000000e-06,0.00,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,Pumpkin,Vegetables,0.015,20.0,4.9,0.000,0.00009,0.07,1.1,0.000009,...,0.0,0.00080,8.000000e-07,0.00,0.0,0.0,0.0,0.0,0.0,0.0
142,Turnip,Vegetables,0.030,28.0,6.4,0.000,0.00009,0.10,1.8,0.000015,...,0.0,0.00003,1.000000e-07,0.00,0.0,0.0,0.0,0.0,0.0,0.0
143,Potato,Vegetables,0.015,93.0,21.0,0.000,0.00012,0.13,2.2,0.000028,...,0.0,0.00004,2.000000e-06,0.00,0.0,0.0,0.0,0.0,0.0,0.0
144,Radicchio,Vegetables,0.019,23.0,4.5,0.000,0.00034,0.25,0.9,0.000060,...,0.0,0.00230,2.550000e-04,0.00,0.0,0.0,0.0,0.0,0.0,0.0


In [466]:
import random

def generate_meal(user_preferences: pd.DataFrame = user_preferences, filtered_df: pd.DataFrame = filtered_df):
    meal = []
    meal.extend([
        random.choice(
            user_preferences[user_preferences["Category Name"] == "Grains"]["Food Name"].to_list() +
            user_preferences[user_preferences["Category Name"] == "Gluten-Free Grains"]["Food Name"].to_list()
        ),
        random.choice(
            user_preferences[user_preferences["Category Name"] == "Legumes"]["Food Name"].to_list() +
            user_preferences[user_preferences["Category Name"] == "Dairy"]["Food Name"].to_list() +
            user_preferences[user_preferences["Category Name"] == "Lactose-Free Dairy"]["Food Name"].to_list() +
            user_preferences[user_preferences["Category Name"] == "Cured Meat"]["Food Name"].to_list() +
            user_preferences[user_preferences["Category Name"] == "Red Meat"]["Food Name"].to_list() +
            user_preferences[user_preferences["Category Name"] == "White Meat"]["Food Name"].to_list() +
            user_preferences[user_preferences["Category Name"] == "Seafood"]["Food Name"].to_list() +
            user_preferences[user_preferences["Category Name"] == "Eggs"]["Food Name"].to_list()
        ),
        random.choice(user_preferences[user_preferences["Category Name"] == "Oils"]["Food Name"].to_list()),
        random.choice(user_preferences[user_preferences["Category Name"] == "Sauces"]["Food Name"].to_list()),
        random.choice(user_preferences[user_preferences["Category Name"] == "Vegetables"]["Food Name"].to_list()),
        random.choice(user_preferences[user_preferences["Category Name"] == "Fruits"]["Food Name"].to_list())
    ])
    
    # Find similar foods for each item in the meal
    similar_meal = []
    for item in meal:
        # Get the category of the current item
        item_category = user_preferences[user_preferences["Food Name"] == item]["Category Name"].values[0]
        
        # Only find similar foods if not a fruit or vegetable
        if item_category not in ["Fruits", "Vegetables", "Greens"]:
            similar_foods = find_similar_food(filtered_df, food_name=item, n=1, same_category=True, low_density_food=True)
            if similar_foods:
                similar_meal.append(similar_foods[0][0])
            else:
                similar_meal.append(item)
        else:
            similar_meal.append(item)
    
    return meal, similar_meal

generate_meal(user_preferences, filtered_df)

(['Wheat Bread',
  'Mortadella',
  'Olive oil',
  'Barbecue sauce',
  'Artichoke',
  'Grapefruit'],
 ['Couscous',
  'Turkey ham',
  'Butter',
  'Tomato sauce',
  'Artichoke',
  'Grapefruit'])

In [467]:
import random

# Load food servings data
with open('../data/raw/food-servings.json', 'r') as f:
    food_servings = json.load(f)

# Define servings count
servings_count = food_servings.copy()

# Generate meals based on frequency per week
generated_meals = []
for category, info in servings_count.items():
    if category in ["Grains", "Gluten-Free Grains", "Fruits", "Vegetables", "Beverages", "Nuts", "Greens", "Oils", "Sauces"]:
        continue
    for _ in range(info["frequency_per_week"]):
        meal = []
        if user_preferences[user_preferences["Category Name"] == category].empty:
            continue
        print(f"Generating {info['frequency_per_week']} meals for {category}...")
        meal.extend([
            random.choice(
                user_preferences[user_preferences["Category Name"] == "Grains"]["Food Name"].to_list() +
                user_preferences[user_preferences["Category Name"] == "Gluten-Free Grains"]["Food Name"].to_list()
            ),
            random.choice(user_preferences[user_preferences["Category Name"] == category]["Food Name"].to_list()),
            random.choice(user_preferences[user_preferences["Category Name"] == "Oils"]["Food Name"].to_list()),
            random.choice(user_preferences[user_preferences["Category Name"] == "Sauces"]["Food Name"].to_list()),
            random.choice(user_preferences[user_preferences["Category Name"] == "Vegetables"]["Food Name"].to_list()),
            random.choice(user_preferences[user_preferences["Category Name"] == "Fruits"]["Food Name"].to_list())
        ])
        
        similar_meal = []
        for item in meal:
            item_category = user_preferences[user_preferences["Food Name"] == item]["Category Name"].values[0]
            if item_category not in ["Oils", "Fruits", "Vegetables", "Greens"]:
                similar_foods = find_similar_food(filtered_df, food_name=item, n=1, same_category=True, low_density_food=True)
                if similar_foods:
                    similar_meal.append(similar_foods[0][0])
                else:
                    similar_meal.append(item)
            else:
                similar_meal.append(item)
        
        generated_meals.append((meal, similar_meal))


    # print(f"Generated {info['frequency_per_week']} meals for {category}.")

# Randomly select 14 meals from the generated meals
if len(generated_meals) < 14:
    lunches = random.sample(generated_meals, len(generated_meals))
    dinners = random.sample(generated_meals, len(generated_meals))

    lunches = lunches[:7]
    dinners = dinners[:7]
else:
    weekly_meals = random.sample(generated_meals, 14)

    # Split the meals into 7 lunches and 7 dinners
    lunches = weekly_meals[:7]
    dinners = weekly_meals[7:]

# Print the generated meals for the week
print("Lunches:")
for day, (meal, similar_meal) in enumerate(lunches, 1):
    print(f"Day {day}:")
    print(f"  Meal: {meal}")
    print(f"  Similar Meal: {similar_meal}")
    print()

print("Dinners:")
for day, (meal, similar_meal) in enumerate(dinners, 1):
    print(f"Day {day}:")
    print(f"  Meal: {meal}")
    print(f"  Similar Meal: {similar_meal}")
    print()

Generating 2 meals for Seafood...
Generating 2 meals for Seafood...
Generating 3 meals for Lactose-Free Dairy...
Generating 3 meals for Lactose-Free Dairy...
Generating 3 meals for Lactose-Free Dairy...
Generating 4 meals for Dairy...
Generating 4 meals for Dairy...
Generating 4 meals for Dairy...
Generating 4 meals for Dairy...
Generating 2 meals for Eggs...
Generating 2 meals for Eggs...
Generating 3 meals for Legumes...
Generating 3 meals for Legumes...
Generating 3 meals for Legumes...
Generating 3 meals for White Meat...
Generating 3 meals for White Meat...
Generating 3 meals for White Meat...
Generating 1 meals for Cured Meat...
Generating 1 meals for Red Meat...
Lunches:
Day 1:
  Meal: ['Wheat Bread', 'Edam', 'Olive oil', 'Mustard', 'Radicchio', 'Kiwifruit']
  Similar Meal: ['Couscous', 'Feta', 'Olive oil', 'Barbecue sauce', 'Radicchio', 'Kiwifruit']

Day 2:
  Meal: ['Rice', 'Chickpeas', 'Olive oil', 'Barbecue sauce', 'Artichoke', 'Kiwifruit']
  Similar Meal: ['Millet', 'Bean', 

In [468]:
def compute_calories_per_meal(meal: list, food_data: pd.DataFrame = df):
    calories = 0
    for food in meal:
        food_info = food_data[food_data['Food Name'] == food]
        if not food_info.empty:
            # consult food-serving.json for serving_size
            food_category = food_data[food_data['Food Name'] == food]['Category Name'].values[0]
            serving_size = food_servings[food_category]['serving_size']
            # print(food_info['Calories'].values[0], serving_size)
            calories += (food_info['Calories'].values[0] * serving_size) / 100
            # print((food_info['Calories'].values[0] * serving_size) / 100)
    return calories

# Compute calories for each meal
lunch_calories = [compute_calories_per_meal(meal) for meal, _ in lunches]
dinner_calories = [compute_calories_per_meal(meal) for meal, _ in dinners]
print("Lunch Calories:", lunch_calories)
print("Dinner Calories:", dinner_calories)
# Sum calories for each day
daily_calories = [lunch + dinner for lunch, dinner in zip(lunch_calories, dinner_calories)]
print("Daily Calories:", daily_calories)

Lunch Calories: [np.float64(928.55), np.float64(641.0999999999999), np.float64(607.4000000000001), np.float64(562.7), np.float64(768.5), np.float64(620.3), np.float64(777.4)]
Dinner Calories: [np.float64(508.09999999999997), np.float64(763.3), np.float64(639.5), np.float64(557.0999999999999), np.float64(980.8), np.float64(751.2), np.float64(468.59999999999997)]
Daily Calories: [np.float64(1436.6499999999999), np.float64(1404.3999999999999), np.float64(1246.9), np.float64(1119.8), np.float64(1749.3), np.float64(1371.5), np.float64(1246.0)]


In [469]:
def generate_breakfast(df):
    breakfast = []
    breakfast.extend([
        random.choice(
            df[df["Category Name"] == "Dairy Breakfast"]["Food Name"].to_list() +
            df[df["Category Name"] == "Lactose-Free Dairy Breakfast"]["Food Name"].to_list() +
            df[df["Category Name"] == "Beverages"]["Food Name"].to_list()
        ),
        random.choice(df[df["Category Name"] == "Baked Products Breakfast"]["Food Name"].to_list()),
        random.choice(
            df[df["Category Name"] == "Sweets Breakfast"]["Food Name"].to_list() +
            df[df["Category Name"] == "Nuts Breakfast"]["Food Name"].to_list()
        ),
        random.choice(df[df["Category Name"] == "Fruits"]["Food Name"].to_list())
    ])
    
    return breakfast

# generate 7 breakfasts
weekly_breakfasts = [generate_breakfast(user_preferences) for _ in range(7)]
print("Weekly Breakfasts:")
for day, meal in enumerate(weekly_breakfasts, 1):
    print(f"Day {day}: {meal}")


Weekly Breakfasts:
Day 1: ['Aloe vera juice', 'White Bread', 'Almond paste', 'Pear']
Day 2: ['Milk', 'White Bread', 'Fruit preserves', 'Kiwifruit']
Day 3: ['Herbal tea', 'Multigrain bread', 'Apricot jam', 'Grapefruit']
Day 4: ['Green tea', 'Biscuit', 'Almond paste', 'Orange']
Day 5: ['Apple juice', 'Bagel', 'Fruit preserves', 'Clementine']
Day 6: ['Cranberry juice', 'Biscuit', 'Peanut butter', 'Orange']
Day 7: ['Orange juice', 'Biscuit', 'Almond paste', 'Grapefruit']


In [470]:
def generate_snacks(df, is_fruit: bool = True):
    if is_fruit:
        category = "Fruits"
    else:
        category = "Nuts"
    
    # Generate the specified number of nuts and fruits
    snack = random.choice(df[df["Category Name"] == category]["Food Name"].to_list())
    
    return snack

# Generate snacks
weekly_snacks = generate_snacks(user_preferences, is_fruit=False)
print("Weekly Fruit Snack:", weekly_snacks)

Weekly Fruit Snack: Chestnut


In [471]:
# Check what day of the week is today
today_day_of_week = datetime.datetime.now().weekday()-1  # Monday is 0 and Sunday is 6

print("Today lunch is:", lunches[today_day_of_week][0])

# Ask the user if today is happy or not
is_happy = input("Is today a happy day? (yes/no): ").lower() == 'yes'

if not is_happy:
    # Change today's lunch with a fast food
    fast_food = random.choice(user_preferences[user_preferences["Category Name"] == "Fast Foods"]["Food Name"].to_list())
    # Clear today's lunch
    new_lunch = []
    new_lunch.append(fast_food)
    del lunches[today_day_of_week]
    similar_foods = find_similar_food(filtered_df, food_name=item, n=1, same_category=False, low_density_food=True)
    if similar_foods:
        (new_lunch, similar_foods[0][0])
    else:
        (new_lunch, new_lunch)
    lunches.insert(today_day_of_week, new_lunch)
    # Add the fast food to today's lunch
    # lunches[today_day_of_week].append(fast_food)

    # Remember that the user has used this "jolly"
    user_input["used_jolly"] = True
else:
    user_input["used_jolly"] = False

if user_input["used_jolly"] == True:
    print(f"It's ok, today's lunch will be: {lunches[today_day_of_week][0]}")

Today lunch is: ['Rice', 'Chickpeas', 'Olive oil', 'Barbecue sauce', 'Artichoke', 'Kiwifruit']


In [472]:
filtered_df.columns

Index(['Food Name', 'Category Name', 'Calcium', 'Calories', 'Carbs',
       'Cholesterol', 'Copper', 'Fats', 'Fiber', 'Folate', 'Iron', 'Magnesium',
       'Monounsaturated Fat', 'Net carbs', 'Omega-3 - DHA', 'Omega-3 - DPA',
       'Omega-3 - EPA', 'Phosphorus', 'Polyunsaturated fat', 'Potassium',
       'Protein', 'Saturated Fat', 'Selenium', 'Sodium', 'Trans Fat',
       'Vitamin A (IU)', 'Vitamin A RAE', 'Vitamin B1', 'Vitamin B12',
       'Vitamin B2', 'Vitamin B3', 'Vitamin B5', 'Vitamin B6', 'Vitamin C',
       'Zinc', 'Choline', 'Fructose', 'Histidine', 'Isoleucine', 'Leucine',
       'Lysine', 'Manganese', 'Methionine', 'Phenylalanine', 'Starch', 'Sugar',
       'Threonine', 'Tryptophan', 'Valine', 'Vitamin D', 'Vitamin E',
       'Vitamin K', 'Omega-3 - ALA', 'Omega-6 - Eicosadienoic acid',
       'Omega-6 - Gamma-linoleic acid', 'Omega-3 - Eicosatrienoic acid',
       'Omega-6 - Dihomo-gamma-linoleic acid', 'Omega-6 - Linoleic acid',
       'Omega-6 - Arachidonic acid'],
   

In [477]:
def find_highest_micronutrient(df: pd.DataFrame, food_name: str):
    micronutrients = [
        "Calcium",
        "Cholesterol",
        "Copper",
        "Folate",
        "Iron",
        "Magnesium",
        "Omega-3 - DHA",
        "Omega-3 - DPA",
        "Omega-3 - EPA",
        "Phosphorus",
        "Potassium",
        "Selenium",
        "Vitamin A (IU)",
        "Vitamin A RAE",
        "Vitamin B1",
        "Vitamin B12",
        "Vitamin B2",
        "Vitamin B3",
        "Vitamin B5",
        "Vitamin B6",
        "Vitamin C",
        "Zinc",
        "Choline",
        "Histidine",
        "Isoleucine",
        "Leucine",
        "Lysine",
        "Manganese",
        "Methionine",
        "Phenylalanine",
        "Threonine",
        "Tryptophan",
        "Valine",
        "Vitamin D",
        "Vitamin E",
        "Vitamin K",
        "Omega-3 - ALA",
        "Omega-6 - Eicosadienoic acid",
        "Omega-6 - Gamma-linoleic acid",
        "Omega-3 - Eicosatrienoic acid",
        "Omega-6 - Dihomo-gamma-linoleic acid",
        "Omega-6 - Linoleic acid",
        "Omega-6 - Arachidonic acid"
    ]
    highest_micronutrient = find_food_nutritional_info(food_name)[micronutrients]
    name = highest_micronutrient.idxmax(axis=1).values[0]
    value = int(df[df['Food Name'] == food_name][name].values[0])
    return name, value

name, value = find_highest_micronutrient(filtered_df, "Apple")
print(f"This food is high in {name}!\nIt has {value} {name} which is good because...")

This food is high in Vitamin A (IU)!
It has 54 Vitamin A (IU) which is good because...


In [478]:
for lunch in lunches:
    for food in lunch:
        for f in food:
            print(f)
            name, value = find_highest_micronutrient(df, f)
            print(f"This food is high in {name}! It has {value} {name}!")

Wheat Bread
This food is high in Potassium! It has 0 Potassium!
Edam
This food is high in Vitamin A (IU)! It has 825 Vitamin A (IU)!
Olive oil
This food is high in Vitamin E! It has 0 Vitamin E!
Mustard
This food is high in Vitamin A (IU)! It has 109 Vitamin A (IU)!
Radicchio
This food is high in Vitamin A (IU)! It has 27 Vitamin A (IU)!
Kiwifruit
This food is high in Vitamin A (IU)! It has 87 Vitamin A (IU)!
Couscous
This food is high in Potassium! It has 0 Potassium!
Feta
This food is high in Vitamin A (IU)! It has 422 Vitamin A (IU)!
Olive oil
This food is high in Vitamin E! It has 0 Vitamin E!
Barbecue sauce
This food is high in Vitamin A (IU)! It has 224 Vitamin A (IU)!
Radicchio
This food is high in Vitamin A (IU)! It has 27 Vitamin A (IU)!
Kiwifruit
This food is high in Vitamin A (IU)! It has 87 Vitamin A (IU)!
Rice
This food is high in Phosphorus! It has 0 Phosphorus!
Chickpeas
This food is high in Vitamin A (IU)! It has 27 Vitamin A (IU)!
Olive oil
This food is high in Vitamin